In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scanpy as sc
import numpy as np
import pandas as pd
from matplotlib.pyplot import rc_context
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 200)
#sc.logging.print_versions() 
sc.logging.print_header()
sc.settings.verbosity = 3  
sc.settings.set_figure_params(dpi=120, color_map='viridis')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.4 scipy==1.9.3 pandas==1.5.1 scikit-learn==1.1.3 statsmodels==0.13.5 python-igraph==0.10.2 pynndescent==0.5.8


In [2]:
adata_gex=sc.read('/home/jovyan/data/covid_19_renal/renal_covid_19_gex.h5ad')

In [4]:
tcell= pd.read_csv('/nfs/team205/ld21/renal/T_NK_compartment_annotation_updated.csv', index_col=0)

In [6]:
adata = adata_gex[(adata_gex.obs.index.isin(tcell.index))]

In [7]:
adata.obs['final_anno']='uk'
adata.obs['final_anno']=adata.obs['final_anno'].astype('object')

/tmp/ipykernel_1517/501034239.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['final_anno']='uk'


In [8]:
adata.obs['final_anno'].update(pd.Series(tcell['level_2'].astype('object')))

In [10]:
adata.obs['final_anno'].unique()

array(['T reg', 'T CD4 Naive', 'T CD4 Helper', 'NK', 'doublets',
       'T CD8 CTL', 'T CD4 CTL', 'NK CD56 bright', 'T CD8 Mem', 'NKT',
       'T CD4 cycling', 'T CD8 Naive', 'MAIT', 'T g/d', 'T CD8 EMRA',
       'ILC'], dtype=object)

In [11]:
adata=adata[~adata.obs['final_anno'].isin(['doublets'])]

In [12]:
list(adata.obs['final_anno'].unique())

['T reg',
 'T CD4 Naive',
 'T CD4 Helper',
 'NK',
 'T CD8 CTL',
 'T CD4 CTL',
 'NK CD56 bright',
 'T CD8 Mem',
 'NKT',
 'T CD4 cycling',
 'T CD8 Naive',
 'MAIT',
 'T g/d',
 'T CD8 EMRA',
 'ILC']

In [14]:
'C139' in list(adata.obs['patient_id'].unique())

True

In [15]:
'C116' in list(adata.obs['patient_id'].unique())

True

In [16]:
'C141' in list(adata.obs['patient_id'].unique())

True

In [18]:
#remove C141 from the samples
adata = adata[~adata.obs['patient_id'].isin(['C141','C139','C116'])]

In [20]:
'C141' in list(adata.obs['patient_id'].unique())

False

In [21]:
'C139' in list(adata.obs['patient_id'].unique())

False

In [22]:
'C116' in list(adata.obs['patient_id'].unique())

False

In [ ]:
#get monocyte only
#monocyte = adata[adata.obs['annot4'].isin(['CD14mono_activated','CD16mono','CD14mono','Int.mono','CD16mono_C1','CD16mono_IFN','CD14mono_IFN','CD14mono_anti_inflammatory'])]

In [23]:
# get all - not only monocyte
monocyte = adata[:]

In [24]:
list(monocyte.obs['final_anno'].unique())

['T reg',
 'T CD4 Naive',
 'NK',
 'T CD4 Helper',
 'T CD8 CTL',
 'T CD4 CTL',
 'NK CD56 bright',
 'T CD4 cycling',
 'T CD8 Naive',
 'MAIT',
 'T g/d',
 'NKT',
 'T CD8 Mem',
 'T CD8 EMRA',
 'ILC']

In [25]:
list(monocyte.obs['case_control'].unique())

['NEGATIVE', 'POSITIVE', 'NA', 'RECOVERY']

In [28]:
#remove 'nan' from case_control
monocyte = monocyte[monocyte.obs['case_control']!='NA']

In [29]:
list(monocyte.obs['case_control'].unique())

['NEGATIVE', 'POSITIVE', 'RECOVERY']

In [30]:
#getting p,n,r in wave 1 and wave 2

monocyte.obs['wave_case_control']='nan'
monocyte.obs.loc[(monocyte.obs["centre"]=='NCL') &(monocyte.obs["case_control"]=='POSITIVE'), "wave_case_control"] = 'wave1_positive'
monocyte.obs.loc[(monocyte.obs["centre"]=='NCL') &(monocyte.obs["case_control"]=='NEGATIVE'), "wave_case_control"] = 'wave1_negative'
monocyte.obs.loc[(monocyte.obs["centre"]=='NCL') &(monocyte.obs["case_control"]=='RECOVERY'), "wave_case_control"] = 'wave1_recovery'

monocyte.obs.loc[(monocyte.obs["centre"]=='Cambridge') &(monocyte.obs["case_control"]=='POSITIVE'), "wave_case_control"] = 'wave2_positive'
monocyte.obs.loc[(monocyte.obs["centre"]=='Cambridge') &(monocyte.obs["case_control"]=='NEGATIVE'), "wave_case_control"] = 'wave2_negative'
monocyte.obs.loc[(monocyte.obs["centre"]=='Cambridge') &(monocyte.obs["case_control"]=='RECOVERY'), "wave_case_control"] = 'wave2_recovery'

/tmp/ipykernel_1517/279369363.py:3: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  monocyte.obs['wave_case_control']='nan'


In [31]:
list(monocyte.obs['wave_case_control'].unique()) #wave 1 doesn't have recovery samples

['wave1_negative',
 'wave1_positive',
 'wave2_positive',
 'wave2_negative',
 'wave2_recovery']

In [32]:
test = monocyte[monocyte.obs['wave_case_control']=='wave2_recovery']
print (list(test.obs['case_control'].unique()))
print (list(test.obs['centre'].unique()))
print (list(test.obs['sample_date'].unique()))

['RECOVERY']
['Cambridge']
['11/03/2021', '25/02/2021', '04/03/2021', '09/03/2021', '10/03/2021']


In [33]:
df = pd.crosstab(monocyte.obs['wave_case_control'],monocyte.obs['final_anno'])
df

final_anno,ILC,MAIT,NK,NK CD56 bright,NKT,T CD4 CTL,T CD4 Helper,T CD4 Naive,T CD4 cycling,T CD8 CTL,T CD8 EMRA,T CD8 Mem,T CD8 Naive,T g/d,T reg
wave_case_control,,,,,,,,,,,,,,,
wave1_negative,41,638,17338,1497,1252,4809,24351,12985,61,19981,220,6624,4912,1183,3090
wave1_positive,56,979,26524,2947,1452,5273,58529,41168,432,20783,115,7256,11534,1655,6886
wave2_negative,12,107,3758,736,224,1406,1886,816,0,4169,6,1622,498,260,45
wave2_positive,49,239,28865,3245,1695,5338,5370,2277,1,22656,52,4836,1108,781,195
wave2_recovery,5,116,5949,762,365,1507,2620,922,2,6032,15,2555,712,256,128


In [ ]:
#df.columns = df.columns.add_categories(['wave_case_control'])
df['wave_case_control'] = df.index # making index into column
df

In [ ]:
p1=df.plot(x='wave_case_control', kind='bar', stacked=True,
        title='distribution of monocytes in Wave 1 and Wave 2')
p1.set_ylabel("num of cells")
p1.set_xlabel(" ")
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  

In [ ]:
#getting percentage
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
df

In [ ]:
df.sum(axis=1)

In [ ]:
monocyte.obs['wave_case_control'].value_counts()

In [ ]:
p2 = df.plot(x='wave_case_control', kind='bar', stacked=True,
        title='distribution of monocytes in Wave1 and Wave2',figsize=(5,5))
p2.set_ylabel('percentage')
p2.set_xlabel('')
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  




p2.text(-0.2,101,'23749',fontsize=9)
p2.text(0.8,101,'22846',fontsize=9)
p2.text(1.8,101,'5364',fontsize=9)
p2.text(2.8,101,'34215',fontsize=9)
p2.text(3.8,101,'6008',fontsize=9)

# Getting wave 2 only 

In [34]:
monocyte_wave1 = monocyte[monocyte.obs['centre']=='NCL']
monocyte_wave2 = monocyte[monocyte.obs['centre']=='Cambridge']

In [35]:
monocyte_wave2.obs['case_control'].unique()

['POSITIVE', 'NEGATIVE', 'RECOVERY']
Categories (3, object): ['NEGATIVE', 'POSITIVE', 'RECOVERY']

In [37]:
# adding steroid status and case_control conditions

monocyte_wave2.obs['steroid_status']='nan'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])) &(monocyte_wave2.obs['case_control']=='POSITIVE'), "steroid_status"] = 'steroid_positive'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])) &(monocyte_wave2.obs['case_control']=='NEGATIVE'), "steroid_status"] = 'steroid_negative'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])) &(monocyte_wave2.obs['case_control']=='RECOVERY'), "steroid_status"] = 'steroid_recovery'

#####################

monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190']))&(monocyte_wave2.obs['case_control']=='POSITIVE'), "steroid_status"] = 'no_steroid_positive'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190']))&(monocyte_wave2.obs['case_control']=='NEGATIVE'), "steroid_status"] = 'no_steroid_negative'


monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190']))&(monocyte_wave2.obs['case_control']=='RECOVERY'), "steroid_status"] = 'no_steroid_recovery'



In [38]:
list(monocyte_wave2.obs['steroid_status'].unique())

['no_steroid_positive',
 'no_steroid_negative',
 'steroid_positive',
 'no_steroid_recovery',
 'steroid_recovery',
 'steroid_negative']

In [39]:
print (list(monocyte_wave2.obs['sample_date'].unique()))

['28/01/2021', '22/01/2021', '16/04/2020', '06/01/2021', '04/02/2021', '02/02/2021', '15/01/2021', '30/01/2021', '11/03/2021', '25/02/2021', '23/01/2021', '22/04/2020', '30/04/2020', '19/01/2021', '27/01/2021', '09/02/2021', '04/03/2021', '12/01/2021', '07/01/2021', '26/01/2021', '20/01/2021', '14/01/2021', '16/01/2021', '09/03/2021', '29/01/2021', '10/02/2021', '12/05/2020', '10/03/2021', '03/02/2021', '11/01/2021', '18/01/2021', '08/02/2021', '01/02/2021', '05/02/2021', '25/01/2021', '26/05/2020', '13/01/2021', '06/02/2021']


In [40]:
test = monocyte_wave2[monocyte_wave2.obs['steroid_status']=='no_steroid_negative']
print (test.obs['case_control'].unique())
print (test.obs['sample_date'].unique())

['NEGATIVE']
Categories (1, object): ['NEGATIVE']
['16/04/2020', '22/04/2020', '30/04/2020', '12/05/2020', '26/05/2020']
Categories (5, object): ['12/05/2020', '16/04/2020', '22/04/2020', '26/05/2020', '30/04/2020']


In [ ]:
test = monocyte_wave2[monocyte_wave2.obs['steroid_status']=='steroid_recovery']
print (test.obs['case_control'].unique())
print (test.obs['sample_date'].unique())
print(test.obs['sample_id_broad'].unique())

In [ ]:
df = pd.crosstab(monocyte_wave2.obs['steroid_status'],monocyte_wave2.obs['annot4'])
df

In [ ]:
#df.columns = df.columns.add_categories(['steroid_status'])
df['steroid_status'] = df.index # making index into column
df

In [ ]:
p1=df.plot(x='steroid_status', kind='bar', stacked=True,
        title='distribution of monocytes in different steroid status')
p1.set_xlabel('')
p1.set_ylabel('number of cells')

plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  

In [ ]:
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
df.sum(axis=1)

In [ ]:
monocyte_wave2.obs['steroid_status'].value_counts()

In [ ]:
p2 = df.plot(x='steroid_status', kind='bar', stacked=True,
        title='distribution of monocytes in different steroid status',figsize=(5,5))
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
p2.set_xlabel('')
p2.set_ylabel('percentage')
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  


p2.text(-0.2,101,'4107',fontsize=9)
p2.text(0.8,101,'21413',fontsize=9)
p2.text(1.8,101,'5033',fontsize=9)
p2.text(2.8,101,'1257',fontsize=9)
p2.text(3.8,101,'12802',fontsize=9)
p2.text(4.8,101,'975',fontsize=9)

# steroid only

In [42]:
# to get sub-samples who got steroid 
steroid = monocyte_wave2[monocyte_wave2.obs['patient_id'].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])]


no_steroid = monocyte_wave2[monocyte_wave2.obs['patient_id'].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190'])]


In [43]:
(list(steroid.obs['case_control'].unique()))

['POSITIVE', 'RECOVERY', 'NEGATIVE']

In [45]:
(list(steroid.obs['patient_id'].unique()))

['C101', 'C138', 'C145', 'C146', 'C147', 'C33']

In [46]:
steroid.obs['discharge_date'].unique()

['21/01/2021', '03/02/2021', '25/01/2021', '26/01/2021', 'NA', '03/06/2021', '11/02/2021']
Categories (7, object): ['03/02/2021', '03/06/2021', '11/02/2021', '21/01/2021', '25/01/2021', '26/01/2021', 'NA']

In [47]:
#correction ##########
steroid.obs['discharge_date'] = steroid.obs['discharge_date'].astype('str')
steroid.obs.loc[steroid.obs["patient_id"] == "C65", "discharge_date"] = '10/06/2020'
steroid.obs['discharge_date'] = steroid.obs['discharge_date'].astype('category')

/tmp/ipykernel_1517/590398472.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  steroid.obs['discharge_date'] = steroid.obs['discharge_date'].astype('str')


In [48]:
tmp = steroid.copy()
tmp.obs['sample_date'] = tmp.obs['sample_date'].astype('object')
tmp.obs['date_first_symptoms'] = tmp.obs['date_first_symptoms'].astype('object')
tmp.obs['date_positive_swab'] = tmp.obs['date_positive_swab'].astype('object')



In [49]:
tmp.obs['date_positive_swab'].unique()

array(['02/01/2021', '16/01/2021', '05/01/2021', '09/01/2021', 'NA',
       '12/01/2021'], dtype=object)

In [50]:
tmp.obs['date_first_symptoms'].replace({'NA':np.nan}, inplace = True)
tmp.obs['date_positive_swab'].replace({'NA':np.nan}, inplace = True)

In [51]:
tmp.obs['date_first_symptoms'].unique()

array(['06/01/2021', '21/01/2021', '05/01/2021', nan, '14/01/2021'],
      dtype=object)

In [52]:
time_from_first_symptoms = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_first_symptoms'], format='%d/%m/%Y')
time_from_positive_swab = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_positive_swab'], format='%d/%m/%Y')

In [53]:
time_from_positive_swab.unique()

array([ 345600000000000, 1641600000000000, 1468800000000000,
       1555200000000000, 1209600000000000, 4060800000000000,
        604800000000000,            'NaT', 1382400000000000,
        259200000000000,  172800000000000,  777600000000000,
        864000000000000,  950400000000000, 4665600000000000,
       1036800000000000,  432000000000000], dtype='timedelta64[ns]')

In [54]:
days_from_first_symptoms = time_from_first_symptoms.dt.days
days_from_positive_swab = time_from_positive_swab.dt.days

In [55]:
days_from_positive_swab.unique()

array([ 4., 19., 17., 18., 14., 47.,  7., nan, 16.,  3.,  2.,  9., 10.,
       11., 54., 12.,  5.])

In [56]:
days_from_first_symptoms.fillna(0, inplace = True)
days_from_positive_swab.fillna(0, inplace = True)

/tmp/ipykernel_1517/315576265.py:1: SettingWithCopyWarning: modifications to a property of a datetimelike object are not supported and are discarded. Change values on the original.
  days_from_first_symptoms.fillna(0, inplace = True)
/tmp/ipykernel_1517/315576265.py:2: SettingWithCopyWarning: modifications to a property of a datetimelike object are not supported and are discarded. Change values on the original.
  days_from_positive_swab.fillna(0, inplace = True)


In [57]:
days_from_positive_swab.unique()

array([ 4., 19., 17., 18., 14., 47.,  7.,  0., 16.,  3.,  2.,  9., 10.,
       11., 54., 12.,  5.])

In [58]:
max(days_from_first_symptoms)


50.0

In [59]:
min(days_from_positive_swab)


0.0

In [60]:
days_from_infection = [max(a,b) for a,b in zip(days_from_first_symptoms, days_from_positive_swab)]


In [61]:
max(days_from_infection)


54.0

In [62]:
steroid.obs['time_from_first_symptoms'] = days_from_first_symptoms
steroid.obs['time_from_positive_swab'] = days_from_positive_swab
steroid.obs['time_from_infection'] = days_from_infection

In [64]:
steroid.obs['final_anno'].unique()

array(['T CD8 Mem', 'T CD4 Helper', 'T CD4 CTL', 'T CD8 CTL',
       'NK CD56 bright', 'NK', 'T CD4 Naive', 'T g/d', 'MAIT', 'T reg',
       'NKT', 'ILC', 'T CD8 Naive', 'T CD8 EMRA'], dtype=object)

In [65]:
# adding dates of steroid inj for each patient
steroid.obs['steroid_date'] = 'nan'
steroid.obs.loc[steroid.obs["patient_id"] == "C146", "steroid_date"] = '22/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C139", "steroid_date"] = '07/02/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C147", "steroid_date"] = '17/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C145", "steroid_date"] = '05/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C138", "steroid_date"] = '28/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C33", "steroid_date"] = '20/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C101", "steroid_date"] = '07/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C141", "steroid_date"] = '02/02/2021'



In [66]:
list(steroid.obs['steroid_date'].unique())

['07/01/2021',
 '28/01/2021',
 '05/01/2021',
 '22/01/2021',
 '17/01/2021',
 '20/01/2021']

In [67]:
tmp = steroid.copy()
tmp.obs['sample_date'] = tmp.obs['sample_date'].astype('object')
time_from_steroid = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['steroid_date'], format='%d/%m/%Y')


In [68]:
days_from_steroid = time_from_steroid.dt.days

In [69]:
days_from_steroid.max()

49

In [70]:
days_from_steroid.unique()

array([  -1,    7,    5,   18,    2,   34,   -5,   14, -273,   11,  -10,
          9,   -3,    1,   -6, -250,    6, -267, -262,    4,   49,   -4,
         13,    0,   -2,   -8])

In [71]:
steroid.obs['days_from_steroid'] = days_from_steroid

In [72]:
steroid.obs['days_from_steroid'].unique()

array([  -1,    7,    5,   18,    2,   34,   -5,   14, -273,   11,  -10,
          9,   -3,    1,   -6, -250,    6, -267, -262,    4,   49,   -4,
         13,    0,   -2,   -8])

In [73]:
# adding metdata about before and after steroid samples based on date of steroid injection
steroid.obs['steroid_timeline']='nan'

steroid.obs.loc[(steroid.obs["days_from_steroid"]<=0)&(steroid.obs['case_control']=='POSITIVE'), "steroid_timeline"] = 'before_steroid_pos'
steroid.obs.loc[(steroid.obs["days_from_steroid"]<=0)&(steroid.obs['case_control']=='NEGATIVE'), "steroid_timeline"] = 'before_steroid_neg'
steroid.obs.loc[(steroid.obs["days_from_steroid"]<=0)&(steroid.obs['case_control']=='RECOVERY'), "steroid_timeline"] = 'before_steroid_rec'

##########################

steroid.obs.loc[(steroid.obs["days_from_steroid"]>0)&(steroid.obs['case_control']=='POSITIVE'), "steroid_timeline"] = 'after_steroid_pos'
steroid.obs.loc[(steroid.obs["days_from_steroid"]>0)&(steroid.obs['case_control']=='NEGATIVE'), "steroid_timeline"] = 'after_steroid_neg'

steroid.obs.loc[(steroid.obs["days_from_steroid"]>0)&(steroid.obs['case_control']=='RECOVERY'), "steroid_timeline"] = 'after_steroid_rec'

steroid.obs['steroid_timeline']=steroid.obs['steroid_timeline'].astype('category')




In [74]:
steroid.obs['steroid_timeline'].unique()

['before_steroid_pos', 'after_steroid_pos', 'after_steroid_rec', 'before_steroid_neg']
Categories (4, object): ['after_steroid_pos', 'after_steroid_rec', 'before_steroid_neg', 'before_steroid_pos']

In [ ]:
'''
# adding metdata about before and after steroid samples based on date of steroid injection
steroid.obs['steroid_timeline1']='nan'

steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_230121','C138_280121','C138_260121','C101_060121',
                                                                     'C141_020221','C141_200121','C141_300121','C141_220121',
                                                                      'C141_260121','C146_120121','C146_160121','C146_140121',
                                                                      'C146_190121','C147_140121','C33_140121','C33_190121','C33_160121']))&(steroid.obs['case_control']=='POSITIVE'), "steroid_timeline1"] = 'before_steroid_pos'
steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_230121','C138_280121','C138_260121','C101_060121',
                                                                     'C141_020221','C141_200121','C141_300121','C141_220121',
                                                                      'C141_260121','C146_120121','C146_160121','C146_140121',
                                                                      'C146_190121','C147_140121','C33_140121','C33_190121','C33_160121']))&(steroid.obs['case_control']=='NEGATIVE'), "steroid_timeline1"] = 'before_steroid_neg'
steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_230121','C138_280121','C138_260121','C101_060121',
                                                                     'C141_020221','C141_200121','C141_300121','C141_220121',
                                                                      'C141_260121','C146_120121','C146_160121','C146_140121',
                                                                      'C146_190121','C147_140121','C33_140121','C33_190121','C33_160121']))&(steroid.obs['case_control']=='RECOVERY'), "steroid_timeline1"] = 'before_steroid_rec'

##########################

steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_040221','C138_020221','C138_300121','C138_300420',
                                                                    'C101_110121','C101_250221','C101_200121','C101_130121',
                                                                    'C101_180121','C141_090321','C145_230121','C145_190121',
                                                                    'C145_070121','C145_140121','C145_120121','C145_300420','C146_250221','C146_270121',
                                                                     'C146_230121','C146_300420','C147_280121','C147_260121','C147_190121','C147_230121','C147_300420',
                                                                     'C33_260121']))&(steroid.obs['case_control']=='POSITIVE'), "steroid_timeline1"] = 'after_steroid_pos'
steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_040221','C138_020221','C138_300121','C138_300420',
                                                                    'C101_110121','C101_250221','C101_200121','C101_130121',
                                                                    'C101_180121','C141_090321','C145_230121','C145_190121',
                                                                    'C145_070121','C145_140121','C145_120121','C145_300420','C146_250221','C146_270121',
                                                                     'C146_230121','C146_300420','C147_280121','C147_260121','C147_190121','C147_230121','C147_300420',
                                                                     'C33_260121']))&(steroid.obs['case_control']=='NEGATIVE'), "steroid_timeline1"] = 'after_steroid_neg'

steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_040221','C138_020221','C138_300121','C138_300420',
                                                                    'C101_110121','C101_250221','C101_200121','C101_130121',
                                                                    'C101_180121','C141_090321','C145_230121','C145_190121',
                                                                    'C145_070121','C145_140121','C145_120121','C145_300420','C146_250221','C146_270121',
                                                                     'C146_230121','C146_300420','C147_280121','C147_260121','C147_190121','C147_230121','C147_300420',
                                                                     'C33_260121']))&(steroid.obs['case_control']=='RECOVERY'), "steroid_timeline1"] = 'after_steroid_rec'




'''

In [ ]:
#test = steroid[steroid.obs['steroid_timeline']=='after_steroid_neg']
#print(test.obs['case_control'].unique())

In [ ]:
df = pd.crosstab(steroid.obs['steroid_timeline'],steroid.obs['annot4'])
df

In [ ]:
#df.columns = df.columns.add_categories(['steroid_timeline'])
df['steroid_timeline'] = df.index # making index into column

In [ ]:
df

In [ ]:
df=df.reindex(['before_steroid_neg', "before_steroid_pos", "after_steroid_pos", "after_steroid_rec"])
df

In [ ]:
p1=df.plot(x='steroid_timeline', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid')
p1.set_ylabel('num of cells')
p1.set_xlabel('')

plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 

In [ ]:
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
steroid.obs['steroid_timeline'].value_counts()

In [ ]:
p2 = df.plot(x='steroid_timeline', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid',figsize=(3,3))
p2.set_xlabel('')
p2.set_ylabel('percentage')
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 


'''
p2.text(-0.2,101,'1257',fontsize=9)
p2.text(0.8,101,'4879',fontsize=9)
p2.text(1.8,101,'7923',fontsize=9)
p2.text(2.8,101,'975',fontsize=9)
'''

# adding metadata

In [ ]:
meta_obs=pd.read_csv('/home/jovyan/data/covid_19_renal/obs_myeloid.csv')

In [ ]:
mdata = sc.read('/home/jovyan/data/covid_19_renal/renal_covid_19_gex_myeloid_soupX_QC.h5ad',cache=True)

In [ ]:
mdata.obs['WHO_temp_severity'] = meta_obs['WHO_temp_severity'].values.astype("str") 
mdata.obs['WHO_temp_severity'] = mdata.obs['WHO_temp_severity'].astype('category')

In [ ]:
steroid.obs['WHO_temp_severity'] = steroid.obs['sex'].astype('object')

In [ ]:
steroid.obs['WHO_temp_severity'].update(pd.Series(mdata.obs['WHO_temp_severity'].astype('object')))

In [ ]:
steroid.obs['WHO_temp_severity'] = steroid.obs['WHO_temp_severity'].astype('category')

In [ ]:
steroid.obs['WHO_temp_severity'].unique()

#  steroid with positive case only -divided into 4 severity 

In [75]:
steroid.obs['steroid_timeline'].unique()

['before_steroid_pos', 'after_steroid_pos', 'after_steroid_rec', 'before_steroid_neg']
Categories (4, object): ['after_steroid_pos', 'after_steroid_rec', 'before_steroid_neg', 'before_steroid_pos']

In [76]:
steroid_pos_only = steroid[steroid.obs['steroid_timeline'].isin(['before_steroid_pos','after_steroid_pos'])]

In [77]:
steroid_pos_only.obs['WHO_temp_severity'].unique()

['severe', 'mild', 'moderate', 'critical']
Categories (4, object): ['critical', 'mild', 'moderate', 'severe']

In [78]:
# adding severity and steroid meta
steroid_pos_only.obs['steroid_severity']='nan'

steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='mild'), "steroid_severity"] = 'before_steroid_pos_mild'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='moderate'), "steroid_severity"] = 'before_steroid_pos_moderate'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='severe'), "steroid_severity"] = 'before_steroid_pos_severe'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='critical'), "steroid_severity"] = 'before_steroid_pos_critical'

steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='mild'), "steroid_severity"] = 'after_steroid_pos_mild'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='moderate'), "steroid_severity"] = 'after_steroid_pos_moderate'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='severe'), "steroid_severity"] = 'after_steroid_pos_severe'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='critical'), "steroid_severity"] = 'after_steroid_pos_critical'


/tmp/ipykernel_1517/1759539440.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  steroid_pos_only.obs['steroid_severity']='nan'


In [79]:
steroid_pos_only.obs['steroid_severity'].unique()

array(['before_steroid_pos_severe', 'after_steroid_pos_mild',
       'after_steroid_pos_moderate', 'before_steroid_pos_moderate',
       'after_steroid_pos_critical', 'before_steroid_pos_mild',
       'after_steroid_pos_severe'], dtype=object)

In [ ]:
#test = steroid_pos_only[steroid_pos_only.obs['steroid_severity']=='after_steroid_pos_severe']
#print(test.obs['case_control'].unique())
#print(test.obs['WHO_temp_severity'].unique())

In [ ]:
df = pd.crosstab(steroid_pos_only.obs['steroid_severity'],steroid_pos_only.obs['annot4'])
df

In [ ]:
df.columns = df.columns.add_categories(['steroid_severity'])
df['steroid_severity'] = df.index 
df

In [ ]:
df=df.reindex(["before_steroid_pos_mild", "after_steroid_pos_mild", "before_steroid_pos_moderate",'after_steroid_pos_moderate','before_steroid_pos_severe','after_steroid_pos_severe',
              'after_steroid_pos_critical'])
df

In [ ]:
p1=df.plot(x='steroid_severity', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid and severity')
p1.set_ylabel('num of cells')
p1.set_xlabel('')

plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 

In [ ]:
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
df

In [ ]:
steroid_pos_only.obs['steroid_severity'].value_counts()

In [ ]:
p2 = df.plot(x='steroid_severity', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid and severity',figsize=(5,5))
p2.set_xlabel('')
p2.set_ylabel('percentage')
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 



p2.text(-0.2,101,'2678',fontsize=9)
p2.text(0.8,101,'1811',fontsize=9)
p2.text(1.8,101,'1911',fontsize=9)
p2.text(2.8,101,'2146',fontsize=9)
p2.text(3.8,101,'290',fontsize=9)
p2.text(4.8,101,'1226',fontsize=9)
p2.text(5.8,101,'2740',fontsize=9)

In [80]:
save_file = '/home/jovyan/scripts/renal_covid_19/steroid_pipeline_corrected/steroid_pos_only_all_T.h5ad'
steroid_pos_only.write_h5ad(save_file)

In [85]:
list(steroid_pos_only.obs['final_anno'].unique())

['T CD8 Mem',
 'T CD4 Helper',
 'T CD4 CTL',
 'T CD8 CTL',
 'NK CD56 bright',
 'NK',
 'T CD4 Naive',
 'T g/d',
 'MAIT',
 'T reg',
 'NKT',
 'ILC',
 'T CD8 Naive',
 'T CD8 EMRA']

# two cat only 

In [ ]:
# adding severity and steroid meta
steroid_pos_only.obs['steroid_severity1']='nan'

steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='mild'), "steroid_severity1"] = 'before_steroid_pos_mild/mod'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='moderate'), "steroid_severity1"] = 'before_steroid_pos_mild/mod'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='severe'), "steroid_severity1"] = 'before_steroid_pos_sev/crit'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='critical'), "steroid_severity1"] = 'before_steroid_pos_sev/crit'

steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='mild'), "steroid_severity1"] = 'after_steroid_pos_mild/mod'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='moderate'), "steroid_severity1"] = 'after_steroid_pos_mild/mod'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='severe'), "steroid_severity1"] = 'after_steroid_pos_sev/crit'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='critical'), "steroid_severity1"] = 'after_steroid_pos_sev/crit'


In [ ]:
steroid_pos_only.obs['steroid_severity1'].unique()

In [ ]:
#test = steroid_pos_only[steroid_pos_only.obs['steroid_severity']=='after_steroid_pos_severe']
#print(test.obs['case_control'].unique())
#print(test.obs['WHO_temp_severity'].unique())

In [ ]:
df = pd.crosstab(steroid_pos_only.obs['steroid_severity1'],steroid_pos_only.obs['annot4'])
df

In [ ]:
df.columns = df.columns.add_categories(['steroid_severity1'])
df['steroid_severity1'] = df.index 
df

In [ ]:
df=df.reindex(["before_steroid_pos_mild/mod", "after_steroid_pos_mild/mod",'before_steroid_pos_sev/crit',
              'after_steroid_pos_sev/crit'])
df

In [ ]:
p1=df.plot(x='steroid_severity1', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid and severity')
p1.set_ylabel('num of cells')
p1.set_xlabel('')

plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 

In [ ]:
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
df

In [ ]:
steroid_pos_only.obs['steroid_severity1'].value_counts()

In [ ]:
p2 = df.plot(x='steroid_severity1', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid and severity',figsize=(5,5))
p2.set_xlabel('')
p2.set_ylabel('percentage')
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 



p2.text(-0.2,101,'2678',fontsize=9)
p2.text(0.8,101,'1811',fontsize=9)
p2.text(1.8,101,'1911',fontsize=9)
p2.text(2.8,101,'2146',fontsize=9)
p2.text(3.8,101,'290',fontsize=9)
p2.text(4.8,101,'1226',fontsize=9)
p2.text(5.8,101,'2740',fontsize=9)

In [ ]:
save_file = '/home/jovyan/scripts/renal_covid_19/steroid_pipeline_corrected/steroid_pipeline/steroid_pos_only.h5ad'
steroid_pos_only.write_h5ad(save_file)

# steroid_pos_rec only

In [ ]:
steroid.obs['steroid_timeline'].unique()

In [ ]:
steroid_pos_rec_only = steroid[steroid.obs['steroid_timeline'].isin(['before_steroid_pos','after_steroid_pos','after_steroid_rec'])]

In [ ]:
steroid_pos_rec_only.obs['steroid_timeline'].unique()

In [ ]:
steroid_pos_rec_only.obs['patient_id'].unique()

In [ ]:
steroid_pos_rec_paired_only = steroid_pos_rec_only[steroid_pos_rec_only.obs['patient_id']!='C145']

In [ ]:
steroid_pos_rec_paired_only.obs['patient_id'].unique()

In [ ]:
save_file = '/home/jovyan/scripts/renal_covid_19/steroid_pipeline_corrected/steroid_pipeline/steroid_pos_rec_only.h5ad'
steroid_pos_rec_only.write_h5ad(save_file)

In [ ]:
save_file = '/home/jovyan/scripts/renal_covid_19/steroid_pipeline_corrected/steroid_pipeline/steroid_pos_rec_paired_only.h5ad'
steroid_pos_rec_paired_only.write_h5ad(save_file)